# Time Series Prediction

This data was sunthesized after reconstructing digitizing a plot from Toronto Hydro PDF then finding the mean and standard deviation fro every time slice in the contructed graph for several days worth of data. This proviided  a gaussian distribution that could be sampled from to generate an infinite number of datas worth of data. Severl such days of synthetic data have been saved in anomaly.csv.

We use this data as a starting point to generate new data in the desired Chronos format: anomalySeries.csv

Then following the tutorial from Article: **Forecasting with Chronos**
- https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-chronos.html



In [ ]:
#!pip install git+https://github.com/amazon-science/chronos-forecasting.git


In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline
from tqdm.auto import tqdm

# Anomaly

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-base",
  device_map="cpu", 
  torch_dtype=torch.bfloat16,
)

In [ ]:
data = TimeSeriesDataFrame(
    "anomalySeries.csv"
)
data.head()

In [ ]:
prediction_length = 24
train_data, test_data = data.train_test_split(prediction_length)

predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(
    train_data, presets="chronos_tiny",
)

In [ ]:
predictor = TimeSeriesPredictor(prediction_length=prediction_length).fit(
    train_data,
    hyperparameters={
        "Chronos": {
            "model_path": "tiny",
            "batch_size": 64,
            "device": "cpu",
        }
    },
    skip_model_selection=True,
    verbosity=0,
)

In [ ]:
predictions = predictor.predict(train_data)

In [ ]:
Len = data.shape[0]
predictor.plot(
    data=data, 
    predictions=predictions, 
    item_ids=["Series"],
    max_history_length=Len,
)

# Generate Clean Anomaly data

In [ ]:
# df = pd.read_csv('anomaly.csv')
# L = []
# for d in range(1,124): # each day for 123 days starting at day 1
#     for h in range(0,24): # each hour
#         for m in range(0,2):  # every 30 min
#             L.append(f"1750-01-{d:02d} {h:02d}:{m*30:02d}:00")
# Len = len(L)
# df = df.loc[:Len-1,['item_id','timestamp','target']]
# df.timestamp = L
# df = df[['item_id','timestamp','target']]
# df.to_csv('anomalySeries.csv', index= False)
# df
